In [13]:
from __future__ import unicode_literals
import spacy
import csv
import re
import numpy as np
import utility as ut
from sklearn import svm
from sklearn.svm import SVC

import features_final as f

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import average_precision_score
from xgboost import XGBClassifier

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [ ]:
def clean(filename,filenameWrite):

    cleaned_data = []
    with open(filename,'r') as file:
        reader = csv.reader(file)
        for row in reader:
            title = row[2]
            description = row[6]
            cleanedDescription = ut.cleanDescription(description)
            #cleanedDescription = cleanText(cleanedDescription)
            cleanedTitle = ut.cleanText(title, False)
            
            # normalize price
            price = float(row[7])
            country = row[0]
            newPrice = ut.normalizePrice(price, country)
            
            cleanedRow = [row[0],row[1],cleanedTitle,row[3],row[4],row[5],cleanedDescription, newPrice,row[8]]
            cleaned_data.append(cleanedRow)

    writer = csv.writer(open(filenameWrite, 'w'))
    writer.writerows(cleaned_data)

In [8]:
w2v_model = f.loadWordVectors('google_news_word_vectors/GoogleNews-vectors-negative300.bin')

Loading Word Vectors............
Loaded Word Vectors.............


In [ ]:
#Can be done only once

print("Cleaning the text in the train data............")
clean('Data/training/data_train.csv','Data/training/data_train_clean.csv')

In [9]:
#Pass the features that you want to test. The order of features are as given below
#word2VecFlag 
#lengthAndBooleanFlag
#tfidfLabelIntersectFlag
#numericFlag
#denseEntropySKUFlag

print("Computing Features............")
X = f.extract_features("Data/training/data_train_clean.csv",w2v_model,False,True,False,False,False)
X_input = X.astype(np.float)

print("Features Computed............")

y_input = np.loadtxt("data/training/conciseness_train.labels", dtype=float)


Computing Features............
Features Computed............


In [10]:
#Normalize the Feature Inputs

X_std = (X_input - X_input.min(axis=0)) / (X_input.max(axis=0) - X_input.min(axis=0))
X_input = X_std
X_input = X_input[:,~np.all(np.isnan(X_input), axis=0)]

In [ ]:
#Pickle the files, load the pickled files
#Use this only if you want to pickle the features and use it later, else don't run this cell

import pickle

#pickle.dump(X_input, open( "X_input_numerical.p", "wb" ))
#pickle.dump(y_input, open( "y_input_numerical.p", "wb" ))

#Load either the X_input_binary or X_input_numerical depending on the features to be tested
#loaded_X_input = pickle.load( open( "X_input_binary.p", "rb" ) )
#loaded_y_input = pickle.load( open( "y_input_binary.p", "rb" ) )

#y_input = np.loadtxt("data/training/clarity_train.labels", dtype=float)

X_input = loaded_X_input

In [ ]:
kFoldResults = ut.splitKFold(X_input,y_input,4)

F1 = []
Acc = []
Precision = []
Roc = []
Recall = []
Avg_Prec = []


for resultGenerator in kFoldResults:
    trainIndices = resultGenerator[0]
    testIndices = resultGenerator[1]
    X_train = list()
    y_train=list()
    X_test=list()
    y_test=list()
    for trainRow in trainIndices:
        X_train.append(X_input[trainRow])
        y_train.append(y_input[trainRow])

    X_train = np.asarray(X_train)
    y_train = np.asarray(y_train)
    
    # Model training
    print("Training the Model............")
    
    
    #Replace with appropriate model, LogisticRegression, RandomForestClassifier etc.. for testing  
    model = XGBClassifier()
    
    model.fit(X_train,y_train)
    
    print("Features Computed for Testing data............")
        
    for testRow in testIndices:
        X_test.append(X_input[testRow])
        y_test.append(y_input[testRow])
        
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    
    y_pred = model.predict(X_test)
    
    y_pred_class = [round(value) for value in y_pred]
       
    # Calculate the performance measures    
    acc = accuracy_score(y_test,y_pred)
    f1 = f1_score(y_test,y_pred )
    roc = roc_auc_score(y_test,y_pred )
    precision = precision_score(y_test, y_pred )
    avg_prec = average_precision_score(y_test, y_pred)
    recall = recall_score(y_test,y_pred)
    
    Acc.append(acc)
    F1.append(f1)
    Roc.append(roc)
    Precision.append(precision)
    Recall.append(recall)
    Avg_Prec.append(avg_prec)
      
    print acc
    print f1 
    print roc
    print precision
    print recall
    print avg_prec

In [ ]:
# Get the average values
print np.mean(Acc)
print np.mean(Roc)
print np.mean(Precision)
print np.mean(F1)
print np.mean(Recall)
print np.mean(avg_prec)